In [2]:
from __future__ import print_function
import sys
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras import backend as K
import matplotlib
import os
import cv2
from google.colab.patches import cv2_imshow
import PIL
import shutil
import time
from PIL import Image
from zipfile import ZipFile
import shutil
import re
import math

In [3]:
def zip_folder(path,name):
  with ZipFile(name, 'w') as zipObj:
  # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(path):
      for filename in filenames:
  #create complete filepath of file in directory
        filePath = os.path.join(folderName, filename)
  # Add file to zip
        zipObj.write(filePath)
  return

In [4]:

# Function for converting decimal to binary 
def float_bin(my_number, places = 3): 
	my_whole, my_dec = str(my_number).split(".") 
	my_whole = int(my_whole) 
	res = (str(bin(my_whole))+".").replace('0b','') 

	for x in range(places): 
		my_dec = str('0.')+str(my_dec) 
		temp = '%1.20f' %(float(my_dec)*2) 
		my_whole, my_dec = temp.split(".") 
		res += my_whole 
	return res 
def IEEE754(n) : 
	# identifying whether the number 
	# is positive or negative 
	sign = 0
	if n < 0 : 
		sign = 1
		n = n * (-1) 
	p = 52
	# convert float to binary 
	dec = float_bin (n, places = p) 

	dotPlace = dec.find('.') 
	onePlace = dec.find('1') 
	# finding the mantissa 
	if onePlace > dotPlace: 
		dec = dec.replace(".","") 
		onePlace -= 1
		dotPlace -= 1
	elif onePlace < dotPlace: 
		dec = dec.replace(".","") 
		dotPlace -= 1
	mantissa = dec[onePlace+1:] 
    
	# calculating the exponent(E) 
	exponent = dotPlace - onePlace 
	exponent_bits = exponent + 1023

	# converting the exponent from 
	# decimal to binary 
	exponent_bits = bin(exponent_bits).replace("0b",'') 

	mantissa = mantissa[0:52]  
	# the IEEE754 notation in binary	 
	final = str(sign) + exponent_bits.zfill(11) + mantissa.zfill(52)

	# convert the binary to hexadecimal 
	
	return (final) 

**0. Loại bỏ cột là các thuộc tính không dùng trước khi up lên**

**1.Up file csv lên (hoặc up lên Google drive cũng được)**
sửa đường dẫn

In [5]:
X=pd.read_csv('/UNSW_NB15_training-set.csv') #path của file train
Y=pd.read_csv('/UNSW_NB15_testing-set.csv') #path của file test

In [6]:
X.columns

Index(['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')

**2. các cột là thuộc tính dùng để tạo ảnh phải liên tiếp nhau trong file csv, chẳng hạn đây là từ 1 đến 42**
Cột 0 không dùng mà e chưa loại bỏ trước

**3. Lưu tạm các cột không dùng, cột label vào biến khác. ở đây attack cat và lable đều là cột chỉ label.**

In [ ]:
Z=X[['attack_cat','label']]
X=X.drop(labels=['attack_cat','label'],axis=1)
V=Y[['attack_cat','label']]
Y=Y.drop(labels=['attack_cat','label'],axis=1)

In [ ]:
frames=[X,Y]
result = pd.concat(frames)

In [ ]:
COLUMN_NAMES = ['feature', 'max', 'min']
df = pd.DataFrame(columns=COLUMN_NAMES)
df

In [ ]:
for i in result.columns:
  if (result[i].dtype!='object'):
    row = pd.DataFrame([[i, result[i].max(), result[i].min()]], columns=COLUMN_NAMES)
    df = df.append(row)

In [ ]:
#       
for i in X.columns:
  if X[i].dtype=='float64':
    max=df[df['feature'] == i]['max'].values[0]
    min=df[df['feature'] == i]['min'].values[0]
    X[i]=X[i].apply(lambda x: (x-min)/(max-min))

In [ ]:
for i in X.columns:
    if (X[i].dtype=='object'):
      X[i]=X[i].astype('category').cat.codes
      X[i]=X[i].apply(lambda x: '{:08b}'.format(x))

In [ ]:
for i in X.columns:
    if (X[i].dtype=='Int64'):
      X[i]=X[i].apply(lambda x: '{:064b}'.format(x))

In [ ]:
for i in X.columns[X.isna().any()].tolist():
    X[i].fillna(-1.0, inplace = True)

for i in X.columns:
    if (X[i].dtype=='float64'):
      X[i]=X[i].apply(lambda x: IEEE754(x))

In [ ]:
#       
for i in Y.columns:
  if Y[i].dtype=='float64':
    max=df[df['feature'] == i]['max'].values[0]
    min=df[df['feature'] == i]['min'].values[0]
    Y[i]=Y[i].apply(lambda x: (x-min)/(max-min))

In [ ]:
for i in Y.columns:
    if (Y[i].dtype=='object'):
      Y[i]=Y[i].astype('category').cat.codes
      Y[i]=Y[i].apply(lambda x: '{:08b}'.format(x))

In [ ]:
for i in Y.columns:
    if (Y[i].dtype=='Int64'):
      Y[i]=Y[i].apply(lambda x: '{:064b}'.format(x))

In [ ]:
for i in Y.columns[Y.isna().any()].tolist():
    Y[i].fillna(-1.0, inplace = True)

for i in Y.columns:
    if (Y[i].dtype=='float64'):
      Y[i]=Y[i].apply(lambda x: IEEE754(x))

**4. Thêm lại cột label vào**

In [ ]:
X['label']=Z['label']
Y['label']=V['label']

**Convert**

**5. ở hàm dưới đây, [1:43] là lấy thuộc tính từ 1 đến 42 (các thuộc tính sử dụng để tạo ảnh), sửa lại cho phù hợp**
**

**6. if row.values[43]==0: kiểm tra cột label để lưu ảnh vào thư mục phù hợp, ở đây 0 là 'normal'**
cần kiểm tra cột label biểu diễn như thế nào

In [ ]:
def convert_to_image(X,path):
  for index, row in X.iterrows():
    joined_features=''.join(row.values[1:43]).replace('\'', '') # nối chuỗi các selected features (từ 1 đến 42)
    image_size=round(math.sqrt(len(joined_features)/8))+1
    arrayA=np.array(re.findall('.{1,8}',joined_features.ljust(image_size*image_size*8,'0')))
    arrayA=vectorized_bin2dec(arrayA)
    arrayA=arrayA.reshape(image_size,image_size)
    arrayA=arrayA.astype(np.uint8)
    image=Image.fromarray(arrayA)
    if row.values[43]==0: #cột 43 là cột label, 0 là normal, 1 là attack
      image.save(path+'/'+'normal'+'/'+str(index)+'.png')
    else:
      image.save(path+'/'+'attack'+'/'+str(index)+'.png')

In [ ]:
bin2dec = lambda i: int(i,2)
vectorized_bin2dec = np.vectorize(bin2dec)

Phải tạo thư mục chứa ảnh trước

In [7]:
os.mkdir('/content/train/')
os.mkdir('/content/train/normal')
os.mkdir('/content/train/attack')

Convert file train

In [ ]:
convert_to_image(X,'/content/train')

In [ ]:
zip_folder('/content/train','Train41frt.zip')

Tiếp theo là file Test

In [ ]:
os.mkdir('/content/val/')
os.mkdir('/content/val/normal')
os.mkdir('/content/val/attack')

In [ ]:
os.mkdir('/content/val/normal')
os.mkdir('/content/val/attack')
convert_to_image(Y,'/content/val')
zip_folder('/content/val','Val41frt.zip')

**7. down 2 file đã nén về máy: Train41frt.zip, Val41frt.zip**